In [1]:
import os

from platform import python_version
import warnings
import time
import datetime as dt
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
import multiprocessing as mp
import shutil

from imblearn.keras import BalancedBatchGenerator
from imblearn.under_sampling import RandomUnderSampler

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.densenet import DenseNet121, preprocess_input, decode_predictions
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.utils import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.initializers import *
from tensorflow.keras import regularizers

import pandas as pd
import numpy as np
import seaborn as sns
from plotly.subplots import make_subplots
import plotly.graph_objects as go

from PIL import Image
import xml.etree.ElementTree as ET
import psutil
import random

warnings.filterwarnings("ignore")
%matplotlib inline

In [2]:
img_size = 96
epochs = 75
batch_size = 32
testsplit = .25
targetx = 96
targety = 96
learning_rate = 0.0001
classes = 7
seed = 23
print(seed)


23


In [3]:
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from collections import Counter

model_name = 'MNV2Balancedmodel.keras'

# Define two different ImageDataGenerators with their respective directories
datagen1 = ImageDataGenerator(
        rescale=1./255,
        brightness_range=[0.9,1.1],
        horizontal_flip=True,
        fill_mode='nearest',
        preprocessing_function=preprocess_input
)

datagen2 = ImageDataGenerator(
        rescale=1./255,
        brightness_range=[0.9,1.1],
        horizontal_flip=True,
        fill_mode='nearest',
        preprocessing_function=preprocess_input
)

train_dir1 = 'KDEFMF/MaleBalancedTraining'
train_dir2 = 'KDEFMF/FemaleBalancedTraining'
val_dir1 = 'KDEFMF/MaleBalancedValidation'
val_dir2 = 'KDEFMF/FemaleBalancedValidation'

# Create two separate generators using different directories
train_generator1 = datagen1.flow_from_directory(
    train_dir1,
    target_size=(targetx,targety),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True,
    seed=seed
)
validation_generator1 = datagen1.flow_from_directory(
    val_dir1,
    target_size=(targetx,targety),
    batch_size=batch_size,
    class_mode='categorical',
)
validation_generator2 = datagen2.flow_from_directory(
    val_dir2,
    target_size=(targetx,targety),
    batch_size=batch_size,
    class_mode='categorical',
)

train_generator2 = datagen2.flow_from_directory(
    train_dir2,
    target_size=(targetx,targety),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True,
    seed=seed
)

total_train_samples = len(train_generator1.filenames) + len(train_generator2.filenames)
total_validation_samples = len(validation_generator1.filenames) + len(validation_generator2.filenames)

# Combine labels from both generators
all_labels = list(train_generator1.classes) + list(train_generator2.classes)

# Get class indices (assuming they are the same for both generators)
class_indices = train_generator1.class_indices
index_to_class = {v: k for k, v in class_indices.items()}

# Count the number of occurrences of each class
class_counts = Counter(all_labels)

# Print the number of images per class
for class_index, count in class_counts.items():
    class_name = index_to_class[class_index]
    print(f"Class '{class_name}' has {count} images.")
    
def combined_generator(gen1, gen2, num_batches):
    while True:
        try:
            batch1_x, batch1_y = next(gen1)
            batch2_x, batch2_y = next(gen2)

            combined_x = np.concatenate((batch1_x, batch2_x), axis=0)
            combined_y = np.concatenate((batch1_y, batch2_y), axis=0)

            yield combined_x, combined_y
        except StopIteration as e:
            print("Generator exhausted or encountered an error:", e)
            break


Found 1120 images belonging to 7 classes.
Found 280 images belonging to 7 classes.
Found 280 images belonging to 7 classes.
Found 1120 images belonging to 7 classes.
Class 'angry' has 320 images.
Class 'disgust' has 320 images.
Class 'fear' has 320 images.
Class 'happy' has 320 images.
Class 'neutral' has 320 images.
Class 'sad' has 320 images.
Class 'surprise' has 320 images.


In [4]:
base_model = MobileNetV2(include_top=False, weights='imagenet', input_shape=(targetx, targety, 3))

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Flatten() (x)
x = BatchNormalization()(x)
x=Dropout(0.3)(x)
x = Dense(512, activation='relu',  kernel_initializer=glorot_uniform(seed),kernel_regularizer=regularizers.L2(0.001),activity_regularizer=regularizers.L2(1e-4), bias_initializer='zeros')(x)
x = BatchNormalization()(x)
x=Dropout(0.3)(x)
x = Dense(256, activation='relu',  kernel_initializer=glorot_uniform(seed),kernel_regularizer=regularizers.L2(0.001),activity_regularizer=regularizers.L2(1e-4), bias_initializer='zeros')(x)
x = BatchNormalization()(x)
x=Dropout(0.3)(x)
x = Dense(128, activation='relu',  kernel_initializer=glorot_uniform(seed),kernel_regularizer=regularizers.L2(0.001),activity_regularizer=regularizers.L2(1e-4), bias_initializer='zeros')(x)
x = BatchNormalization()(x)
x=Dropout(0.3)(x)
predictions = Dense(classes, activation='softmax', kernel_initializer='random_uniform', bias_initializer='zeros')(x)

model = tf.keras.Model(inputs=base_model.input, outputs=predictions)
optimizer = Adam(learning_rate=learning_rate)
loss = "categorical_crossentropy"

for layer in model.layers:
    layer.trainable = True

model.compile(optimizer=optimizer,
              loss=loss,
              metrics=["accuracy"])


In [5]:
# Define the number of batches you want from each generator
num_batches_per_gen = 100  # Adjust as needed

# Use the combined generator for training
combined_train_generator = combined_generator(train_generator1, train_generator2, num_batches_per_gen)

# Use the combined generator for validation
combined_validation_generator = combined_generator(validation_generator1, validation_generator2, num_batches_per_gen)


steps_per_epoch = total_train_samples // (2 * 32)
validation_steps = total_validation_samples // (2 * 32)

# Ensure correct optimizer and loss function
optimizer = Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Ensure ModelCheckpoint is correctly configured used for callback to save the best value based on val_accuracy
checkpoint = ModelCheckpoint(model_name, monitor='val_accuracy', save_best_only=True, verbose=1)

# Train the model with callbacks
history = model.fit(
    combined_train_generator,
    steps_per_epoch=steps_per_epoch,
    validation_data=combined_validation_generator,
    validation_steps=validation_steps,
    epochs=epochs,
    callbacks=[checkpoint]
)

Epoch 1/75
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 541ms/step - accuracy: 0.1507 - loss: 8.7734
Epoch 1: val_accuracy improved from -inf to 0.14648, saving model to MNV2Balancedmodel.keras
35/35 ━━━━━━━━━━━━━━━━━━━━ 56s 700ms/step - accuracy: 0.1511 - loss: 8.7693 - val_accuracy: 0.1465 - val_loss: 5.5690
Epoch 2/75
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 504ms/step - accuracy: 0.2080 - loss: 8.1850
Epoch 2: val_accuracy did not improve from 0.14648
35/35 ━━━━━━━━━━━━━━━━━━━━ 20s 573ms/step - accuracy: 0.2081 - loss: 8.1819 - val_accuracy: 0.1411 - val_loss: 5.5803
Epoch 3/75
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 499ms/step - accuracy: 0.2186 - loss: 7.7401
Epoch 3: val_accuracy did not improve from 0.14648
35/35 ━━━━━━━━━━━━━━━━━━━━ 19s 551ms/step - accuracy: 0.2187 - loss: 7.7371 - val_accuracy: 0.1431 - val_loss: 5.6287
Epoch 4/75
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 469ms/step - accuracy: 0.2544 - loss: 7.2984
Epoch 4: val_accuracy improved from 0.14648 to 0.15927, saving model to MNV2Balancedmodel.keras
35/35 ━━━━━

In [6]:
# Collect predictions and true labels
y_true = []
y_pred = []

for batch_x, batch_y in combined_validation_generator:
    # Make predictions
    predictions = model.predict(batch_x)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(batch_y, axis=1)
    
    # Append to lists
    y_true.extend(true_classes)
    y_pred.extend(predicted_classes)
    
    # Stop if you have processed the entire validation set
    if len(y_true) >= total_validation_samples:
        break

# Generate classification report
report = classification_report(y_true, y_pred, target_names=validation_generator1.class_indices.keys())
print(report)

2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
              precision    recall  f1-score   support

       angry       0.57      0.05      0.09        83
     disgust       0.14      0.78      0.23        81
        fear       0.00      0.00      0.00        84
       happy       0.00      0.00      0.00        78
     neutral       0.00      0.00      0.00        76
         sad       1.00      0.01      0.02        87
    surprise       0.11      0.14      0.12        71

    accuracy                           0.14       560
   macro avg       0.26      0.14      0.07       560
weighted avg       0.27      0.14      0.07       560



In [7]:
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report

# Assuming you have the model and validation generators defined
# validation_generator1 and validation_generator2
# Also assuming the target_names are the same for both generators

# Function to generate and print classification report
def generate_classification_report(generator, generator_name):
    y_true = []
    y_pred = []

    for batch_x, batch_y in generator:
        # Generate predictions
        predictions = model.predict(batch_x)
        predicted_classes = np.argmax(predictions, axis=1)
        true_classes = np.argmax(batch_y, axis=1)

        # Collect predictions and true labels
        y_true.extend(true_classes)
        y_pred.extend(predicted_classes)

        # Break if processed all samples
        if len(y_true) >= generator.samples:
            break

    # Generate and print the classification report
    target_names = list(generator.class_indices.keys())
    report = classification_report(y_true, y_pred, target_names=target_names)
    print(f"Classification Report for {generator_name}:\n{report}")

# Generate and print classification reports for both validation generators
generate_classification_report(validation_generator1, "Validation Generator 1")
generate_classification_report(validation_generator2, "Validation Generator 2")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
Classification Report for Validation Generator 1:
              precision    recall  f1-score   support

       angry       0.00      0.00      0.00        36
     disgust       0.14      0.82      0.23        38
        fear       0.00      0.00      0.00        39
       happy       0.00      0.00      0.00        39
     neutral       0.00      0.00      0.00        45
         sad       0.00      0.00      0.00        38
    surprise       0.13      0.13      0.13        45

    accuracy                           0.13       280
   macro avg       0.04      0.14      0.05       280
weighted avg       0.04      0.13      0.05       280

1/1 ━━━━━━━

In [8]:
print(model_name)

MNV2Balancedmodel.keras
